**Creación de la base de datos de Personajes Mitológicos scrapeados de Wikipedia**

1. Obtención del dataframe con todas las mitologías disponibles en Wikipedia.

#1. Obtención de las URLs de cada categoría que se quiere scrapear.
#2. Obtención de las URLs de los personajes.
#3. Una vez limpio el dataset con los personajes, se obtiene el texto de Wikipedia. También se obtiene la matriz de adyacencia de los links de los personajes.

In [1]:
import pandas as pd

import time

import re
import unicodedata

import requests
from bs4 import BeautifulSoup

Función para obtener el objeto *BeautifulSoup* de la url.

In [2]:
def get_url(url):
    req = requests.get(url)
    if req.status_code == 200:
        document = BeautifulSoup(req.text, "html.parser")
    else:
        document = None
    return document

## 1. Mitologías por región

Se crea un dataframe con las mitologías que se scrapearán y la región a la que pertenecen.

### Obtención de las mitologías

El conjunto de mitologías se obtiene de diferentes formas:

1. [Hijos de la categoría "Mitología por cultura"](https://es.wikipedia.org/wiki/Especial:%C3%81rbolDeCategor%C3%ADas?target=Mitolog%C3%ADa_por_cultura&mode=all)
2. [Descendientes de la categoría "Mitología por continente"](https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente)
3. [Anexo de deidades](https://es.wikipedia.org/wiki/Anexo:Deidades)
4. Mitologías añadidas directamente a la lista

#### Mitologías 1

In [3]:
url = "https://es.wikipedia.org/wiki/Especial:ÁrbolDeCategorías?target=Mitología_por_cultura&mode=all"

pag = get_url(url)
nodos = pag.find("div", class_ = "CategoryTreeChildren").contents

mitologias1 = []
for nodo in nodos[3:]: # los primeros 3 elementos no contienen mitologias
    mitologias1.append(nodo.find("a").text)

In [4]:
mitologias1

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

#### Mitologías 2

Código válido a día 30/08/2021

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Sergi\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [6]:
url = "https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente"

driver.get(url)

In [7]:
def es_mitologia(mit_name):
    return(mit_name[:10] == "Mitología " and " en " not in mit_name)

In [8]:
def en_lista(mit_name, lista):
    return(mit_name in lista)

In [9]:
def expand_mitologia(mit_name):
    try:
        state = (driver
                 .find_element_by_xpath("//span[@data-ct-title='{}']"
                                        .format(mit_name
                                                .replace(" ", "_")))
                 .get_attribute("title"))
    except:
        return(False)
    return(state == "expandir")

In [10]:
mitologias2 = []

nodos = (driver
         .find_element_by_xpath("//div[@class='mw-content-ltr']")
         .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))

while nodos != []:
    
    time.sleep(0.2)
    nombre_mitologia = nodos[0].text[:(nodos[0].text.find("(") - 1)]
    
    # si no es mitología o ya está en la lista...
    if (not(es_mitologia(nombre_mitologia)) or en_lista(nombre_mitologia, mitologias2)):
        del nodos[0]
        continue
    
    # si es mitología y no está en la lista...
    mitologias2.append(nombre_mitologia)
    if expand_mitologia(nombre_mitologia):
        (driver
         .find_element_by_xpath("//span[@data-ct-title='{}']"
                                .format(nombre_mitologia)
                                .replace(" ", "_"))
         .click())
        nodos = (driver
                 .find_element_by_xpath("//div[@class='mw-content-ltr']")
                 .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))

In [25]:
from itertools import compress

mitologias2 = list(compress(mitologias2, [" por " not in m for m in mitologias2]))

In [26]:
mitologias2

['Mitología de África',
 'Mitología de América',
 'Mitología bereber',
 'Mitología cartaginesa',
 'Mitología egipcia',
 'Mitología guanche',
 'Mitología igbo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología masai',
 'Mitología pigmea',
 'Mitología yoruba',
 'Mitología zulú',
 'Mitología del Caribe',
 'Mitología de América Central',
 'Mitología anishinaabe',
 'Mitología de América del Norte',
 'Mitología afroamericana',
 'Mitología de Costa Rica',
 'Mitología de El Salvador',
 'Mitología talamanqueña',
 'Mitología de Guatemala',
 'Mitología bribri',
 'Mitología mesoamericana',
 'Mitología taína',
 'Mitología canadiense',
 'Mitología mexica',
 'Mitología maya',
 'Mitología tlaxcalteca',
 'Mitología mixteca',
 'Mitología olmeca',
 'Mitología purépecha',
 'Mitología tolteca',
 'Mitología inuit',
 'Mitología circumpolar',
 'Mitología estadounidense',
 'Mitología ainu',
 'Mitología hopi',
 'Mitología pueblo',
 'Mitología mexicana',
 'Mitología de América del Sur',
 'Mitología de As

#### Mitologías 3

In [27]:
url = "https://es.wikipedia.org/wiki/Anexo:Deidades"

pag = get_url(url)
items = [item.text for item in pag.find("div", id = "toc").find_all("span", class_ = "toctext")]

mitologias3 = []
for item in items:
    if item[:10] == "Mitología ":
        mitologias3.append(item)

In [28]:
mitologias3

['Mitología abenaki',
 'Mitología aborigen australiana',
 'Mitología judaica',
 'Mitología del cristianismo',
 'Mitología del catolicismo',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología árabe',
 'Mitología ashanti',
 'Mitología azteca',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología budista',
 'Mitología bushongo',
 'Mitología celta',
 'Mitología china',
 'Mitología chippewa',
 'Mitología coreana',
 'Mitología creek',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología dinka',
 'Mitología efik',
 'Mitología egipcia',
 'Mitología escandinava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología frigia',
 'Mitología griega',
 'Mitología guanche',
 'Mitología guaraní',
 'Mitología haida',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología huron',
 'Mitología igbo',
 'Mitología inca',
 'Mitología inuit',
 'Mitología iroquesa',
 'Mitología isoko',
 'Mitología japonesa',
 'Mitología khoikhoi',
 'Mitología kongo',
 'Mitología kurumba',
 'Mito

#### Mitologías 4

Las anteriores listas no recogen todas las mitologías de Wikipedia. Se añaden 13 mitologías fuera de la lista.

In [29]:
mitologias4 = ["Mitología africana", "Mitología americana", "Mitología asiática", "Mitología europea", "Mitología oceánica",
               "Mitología semítica", "Mitología nubia", "Mitología del hinduismo", "Mitología judía", "Mitología vudú",
               "Mitología hawaiana", "Mitología samoana", "Mitología tahitiana"]

#### Agregación de las mitologías

In [30]:
lista_mitologias = list(pd.unique(mitologias1 + mitologias2 + mitologias3 + mitologias4))

lista_mitologias

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

### Mitologías por región

Se pueden encontrar algunas mitologías en inglés con sus respectivas regiones. Las mitologías restantes sin región se añaden a mano.

#### Obtención en inglés

In [33]:
url = "https://en.wikipedia.org/wiki/List_of_mythologies"

pagina = get_url(url)
items = pagina.find("div", class_ = "mw-parser-output").contents

items_clean = []
for item in items:
    if not(isinstance(item, str)) and item.name in ["h2", "h3", "h4", "ul"]:
        items_clean.append(item)
items_clean.pop(0)

<h2><span class="mw-headline" id="Mythologies_by_region">Mythologies by region</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_mythologies&amp;action=edit&amp;section=1" title="Edit section: Mythologies by region">edit</a><span class="mw-editsection-bracket">]</span></span></h2>

In [34]:
def clean_edit(string):
    if string[-6:] == "[edit]":
        return(string[:-6])
    else:
        return(string)

In [35]:
mythologies = pd.DataFrame()
h2 = True
i = 0
while h2:
    if items_clean[i].name == "h2":
        h2 = False
    elif items_clean[i].name == "h3":
        h3 = clean_edit(items_clean[i].text)
        h4 = None
    elif items_clean[i].name == "h4":
        h4 = clean_edit(items_clean[i].text)
    elif items_clean[i].name == "ul":
        myth_split = items_clean[i].text.split("\n")
        for item in myth_split:
            if item[-9:] == "mythology":
                mythologies = mythologies.append(pd.DataFrame([[item, h4, h3]]))
    i += 1

mythologies = mythologies.reset_index(drop = True)
mythologies.columns = ["Mythology", "Region", "Continent"]

mythologies

,Mythology,Region,Continent
0,Baluba mythology,Central Africa,Africa
1,Bushongo mythology,Central Africa,Africa
2,Kongo mythology,Central Africa,Africa
3,Lugbara mythology,Central Africa,Africa
4,Mbuti mythology,Central Africa,Africa
...,...,...,...
142,Chilote mythology,South America,Americas
143,Guarani mythology,South America,Americas
144,Inca mythology,South America,Americas
145,Mapuche mythology,South America,Americas


#### Unión en español

In [36]:
mitologias_esp = [None] * len(mythologies)
url_base = "https://en.wikipedia.org/wiki/{}"
for i in range(len(mythologies)):
    url = url_base.format(mythologies["Mythology"][i].replace(" ", "_"))
    page = get_url(url)
    if page is None:
        continue
    es_link = page.find("a", hreflang = "es", class_ = "interlanguage-link-target")
    if es_link is not None:
        pos = es_link.get("title").find(" – Spanish")
        mitologias_esp[i] = es_link.get("title")[:pos]

mythologies["Mitología"] = mitologias_esp

del mythologies["Mythology"]

mythologies = mythologies[mythologies["Mitología"].isin(lista_mitologias)]

In [37]:
mythologies

,Region,Continent,Mitología
3,Central Africa,Africa,Mitología lugbara
11,North Africa,Africa,Mitología egipcia
28,Northeast Asia,Asia,Mitología china
29,Northeast Asia,Asia,Mitología japonesa
30,Northeast Asia,Asia,Mitología coreana
37,Northeast Asia,Asia,Mitología túrquica
38,South Asia,Asia,Mitología budista
39,South Asia,Asia,Mitología hinduista
53,West Asia / Middle East,Asia,Mitología cristiana
55,West Asia / Middle East,Asia,Mitología judía


Falta añadir la región de las mitologías siguientes. A pesar de que sean muchas, considero que la región es una variable suficientemente importante para añadirla a mano.

In [42]:
set(lista_mitologias) - set(mythologies["Mitología"])

{'Mitología Báltica',
 'Mitología abenaki',
 'Mitología abrahámica',
 'Mitología acadia',
 'Mitología africana',
 'Mitología afroamericana',
 'Mitología aimara',
 'Mitología ainu',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología amazónica',
 'Mitología americana',
 'Mitología anatolia',
 'Mitología anglosajona',
 'Mitología anishinaabe',
 'Mitología aragonesa',
 'Mitología argentina',
 'Mitología ashanti',
 'Mitología asiria',
 'Mitología asiática',
 'Mitología asturiana',
 'Mitología azteca',
 'Mitología babilónica',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología bereber',
 'Mitología boliviana',
 'Mitología bribri',
 'Mitología bushongo',
 'Mitología canadiense',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología castellana',
 'Mitología catalana',
 'Mitología chibcha',
 'Mitología chilena',
 'Mitología chippewa',
 'Mitología circumpolar',
 'Mitología colombiana',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología de América',
 'Mitolog

In [39]:
addMyths = [["Northern Europe", "Europe", "Mitología Báltica"],
            ["North America", "Americas", "Mitología abenaki"],
            ["West Asia / Middle East", "Asia", "Mitología abrahámica"],
            ["West Asia / Middle East", "Asia", "Mitología acadia"],
            [None, "Africa", "Mitología africana"],
            ["African Diasporic", "Africa", "Mitología afroamericana"],
            ["South America", "Americas", "Mitología aimara"],
            ["North America", "Americas", "Mitología ainu"],
            ["East Africa", "Africa", "Mitología akamba"],
            ["West Africa", "Africa", "Mitología akan"],
            ["Central Africa", "Africa", "Mitología alur"],
            ["South America", "Americas", "Mitología amazónica"],
            [None, "Americas", "Mitología americana"],
            ["West Asia / Middle East", "Asia", "Mitología anatolia"],
            ["Northern Europe", "Europe", "Mitología anglosajona"],
            ["Central America", "Americas", "Mitología anishinaabe"],
            ["Western Europe", "Europe", "Mitología aragonesa"],
            ["South America", "Americas", "Mitología argentina"],
            ["West Africa", "Africa", "Mitología ashanti"],
            ["West Asia / Middle East", "Asia", "Mitología asiria"],
            [None, "Asia", "Mitología asiática"],
            ["Western Europe", "Europe", "Mitología asturiana"],
            ["Central America", "Americas", "Mitología azteca"],
            ["West Asia / Middle East", "Asia", "Mitología babilónica"],
            ["East Africa", "Africa", "Mitología baganda"],
            ["East Africa", "Africa", "Mitología banyarwanda"],
            ["North Africa", "Africa", "Mitología bereber"],
            ["South America", "Americas", "Mitología boliviana"],
            ["Central America", "Americas", "Mitología bribri"],
            ["Central Africa", "Africa", "Mitología bushongo"],
            ["North America", "Americas", "Mitología canadiense"],
            ["West Asia / Middle East", "Asia", "Mitología cananea"],
            ["North Africa", "Africa", "Mitología cartaginesa"],
            ["Western Europe", "Europe", "Mitología castellana"],
            ["Western Europe", "Europe", "Mitología catalana"],
            ["South America", "Americas", "Mitología chibcha"],
            ["South America", "Americas", "Mitología chilena"],
            ["North America", "Americas", "Mitología chippewa"],
            ["North America", "Americas", "Mitología circumpolar"],
            ["South America", "Americas", "Mitología colombiana"],
            ["Eastern Europe", "Europe", "Mitología dacia"],
            ["West Africa", "Africa", "Mitología dahomey"],
            [None, "Americas", "Mitología de América"],
            ["Central America", "Americas", "Mitología de América Central"],
            ["North America", "Americas", "Mitología de América del Norte"],
            ["South America", "Americas", "Mitología de América del Sur"],
            ["Southern Europe", "Europe", "Mitología de Argólida"],
            [None, "Asia", "Mitología de Asia"],
            ["Northeast Asia", "Asia", "Mitología de Asia Oriental"],
            ["Central America", "Americas", "Mitología de Costa Rica"],
            ["Southern Europe", "Europe", "Mitología de Creta"],
            ["Central America", "Americas", "Mitología de El Salvador"],
            [None, "Europe", "Mitología de Europa"],
            ["Oceania", "Oceania", "Mitología de Fiyi"],
            ["Western Europe", "Europe", "Mitología de Galicia"],
            ["Southern Europe", "Europe", "Mitología de Grecia Central"],
            ["Central America", "Americas", "Mitología de Guatemala"],
            ["Eastern Europe", "Europe", "Mitología de Hungría"],
            ["West Asia / Middle East", "Asia", "Mitología de Levante"],
            ["Oceania", "Oceania", "Mitología de Melanesia"],
            ["Oceania", "Oceania", "Mitología de Micronesia"],
            ["Oceania", "Oceania", "Mitología de Oceanía"],
            ["West Asia / Middle East", "Asia", "Mitología de Oriente Medio"],
            ["Oceania", "Oceania", "Mitología de Polinesia"],
            ["Eastern Europe", "Europe", "Mitología de Rumania"],
            [None, "Africa", "Mitología de África"],
            ["Southern Europe", "Europe", "Mitología de Élide"],
            ["West Asia / Middle East", "Asia", "Mitología del Antiguo Oriente Próximo"],
            ["Central Asia", "Asia", "Mitología del Asia Central"],
            ["South Asia", "Asia", "Mitología del Asia meridional"],
            ["Northeast Asia", "Asia", "Mitología del Asia norte"],
            ["Southeast Asia", "Asia", "Mitología del Asia sureste"],
            ["Central America", "Americas", "Mitología del Caribe"],
            ["Western Europe", "Europe", "Mitología del País Vasco"],
            ["Southern Europe", "Europe", "Mitología del Peloponeso"],
            ["Western Europe", "Europe", "Mitología del catolicismo"],
            ["West Asia / Middle East", "Asia", "Mitología del cristianismo"],
            ["South Asia", "Asia", "Mitología del hinduismo"],
            ["West Asia / Middle East", "Asia", "Mitología del islam"],
            ["West Asia / Middle East", "Asia", "Mitología del judaísmo"],
            ["North Africa", "Africa", "Mitología dinka"],
            ["Southern Europe", "Europe", "Mitología doriana"],
            ["West Africa", "Africa", "Mitología efik"],
            ["Northern Europe", "Europe", "Mitología escandinava"],
            ["Eastern Europe", "Europe", "Mitología eslava"],
            ["Western Europe", "Europe", "Mitología española"],
            ["North America", "Americas", "Mitología estadounidense"],
            [None, "Europe", "Mitología europea"],
            ["Western Europe", "Europe", "Mitología extremeña"],
            ["West Asia / Middle East", "Asia", "Mitología fenicia"],
            ["Southeast Asia", "Asia", "Mitología filipina"],
            ["West Asia / Middle East", "Asia", "Mitología filistea"],
            ["Northern Europe", "Europe", "Mitología finlandesa"],
            ["Western Europe", "Europe", "Mitología francesa"],
            ["West Asia / Middle East", "Asia", "Mitología frigia"],
            ["Western Europe", "Europe", "Mitología gala"],
            ["West Asia / Middle East", "Asia", "Mitología griega de Asia Menor"],
            ["Southern Europe", "Europe", "Mitología griega de Chipre"],
            ["Southern Europe", "Europe", "Mitología griega de Corinto"],
            ["Southern Europe", "Europe", "Mitología griega de Sicilia"],
            ["Southern Europe", "Europe", "Mitología griega de Tesalia"],
            ["Southern Europe", "Europe", "Mitología griega de Tracia"],
            ["Southern Europe", "Europe", "Mitología griega de Épiro"],
            ["West Africa", "Africa", "Mitología guanche"],
            ["North America", "Americas", "Mitología haida"],
            ["West Asia / Middle East", "Asia", "Mitología hattiana"],
            ["Oceania", "Oceania", "Mitología hawaiana"],
            ["South America", "Americas", "Mitología huilliche"],
            ["North America", "Americas", "Mitología huron"],
            ["West Asia / Middle East", "Asia", "Mitología hurrita"],
            ["West Africa", "Africa", "Mitología igbo"],
            ["South America", "Americas", "Mitología inca"],
            ["Eastern Europe", "Europe", "Mitología indoeuropea"],
            ["North America", "Americas", "Mitología iroquesa"],
            ["West Africa", "Africa", "Mitología isoko"],
            ["Southern Europe", "Europe", "Mitología itálica"],
            ["West Asia / Middle East", "Asia", "Mitología judaica"],
            ["Southern Africa", "Africa", "Mitología khoikhoi"],
            ["Central Africa", "Africa", "Mitología kongo"],
            ["West Africa", "Africa", "Mitología kurumba"],
            ["North Africa", "Africa", "Mitología kushita"],
            ["North America", "Americas", "Mitología kwakiutl"],
            ["Southern Europe", "Europe", "Mitología laconia"],
            ["North America", "Americas", "Mitología lakota"],
            ["Northern Europe", "Europe", "Mitología lapona"],
            ["Northern Europe", "Europe", "Mitología letona"],
            ["West Asia / Middle East", "Asia", "Mitología levantina"],
            ["North Africa", "Africa", "Mitología lotuko"],
            ["East Africa", "Africa", "Mitología malgache"],
            ["South America", "Americas", "Mitología mapuche"],
            ["East Africa", "Africa", "Mitología masai"],
            ["Central America", "Americas", "Mitología mesoamericana"],
            ["West Asia / Middle East", "Asia", "Mitología mesopotámica"],
            ["Central America", "Americas", "Mitología mexicana"],
            ["Central America", "Americas", "Mitología mixteca"],
            ["North America", "Americas", "Mitología navajo"],
            ["North Africa", "Africa", "Mitología nubia"],
            ["Oceania", "Oceania", "Mitología oceánica"],
            ["North America", "Americas", "Mitología pawnee"],
            ["South America", "Americas", "Mitología peruana"],
            ["Central Africa", "Africa", "Mitología pigmea"],
            ["Western Europe", "Europe", "Mitología portuguesa"],
            ["North America", "Americas", "Mitología pueblo"],
            ["Central America", "Americas", "Mitología purépecha"],
            ["South America", "Americas", "Mitología quechua"],
            ["Eastern Europe", "Europe", "Mitología rusa"],
            ["Northern Europe", "Europe", "Mitología sajona y anglosajona"],
            ["North America", "Americas", "Mitología salish"],
            ["Southern Europe", "Europe", "Mitología sarda"],
            ["South America", "Americas", "Mitología selknam"],
            ["West Asia / Middle East", "Asia", "Mitología semítica"],
            ["Northeast Asia", "Asia", "Mitología shinto"],
            ["Northeast Asia", "Asia", "Mitología siberiana"],
            ["West Asia / Middle East", "Asia", "Mitología sumeria"],
            ["North America", "Americas", "Mitología séneca"],
            ["Oceania", "Oceania", "Mitología tahitiana"],
            ["Central America", "Americas", "Mitología taína"],
            ["Central America", "Americas", "Mitología tlaxcalteca"],
            ["Central America", "Americas", "Mitología tolteca"],
            ["Southern Europe", "Europe", "Mitología tracia"],
            ["East Africa", "Africa", "Mitología tumbuka"],
            ["Eastern Europe", "Europe", "Mitología ucraniana"],
            ["West Asia / Middle East", "Asia", "Mitología ugaritiche"],
            ["South America", "Americas", "Mitología venezolana"],
            ["African Diasporic", "Africa", "Mitología vudú"],
            ["North America", "Americas", "Mitología winnebago"],
            ["Central America", "Americas", "Mitología wixarrica"],
            ["West Africa", "Africa", "Mitología yoruba"],
            ["Southern Africa", "Africa", "Mitología zulú"],
            ["North America", "Americas", "Mitología zuñi"],
            ["Southern Europe", "Europe", "Mitología ática"],
            ["Western Europe", "Europe", "Mitología íbera"]]

In [43]:
mitologias = (mythologies
              .append(pd.DataFrame(addMyths, columns = ["Region", "Continent", "Mitología"]))
              .drop_duplicates()
              .set_index("Mitología"))

mitologias

,Region,Continent
Mitología,,
Mitología lugbara,Central Africa,Africa
Mitología egipcia,North Africa,Africa
Mitología china,Northeast Asia,Asia
Mitología japonesa,Northeast Asia,Asia
Mitología coreana,Northeast Asia,Asia
...,...,...
Mitología yoruba,West Africa,Africa
Mitología zulú,Southern Africa,Africa
Mitología zuñi,North America,Americas


In [44]:
mitologias.to_csv("mitologias.csv", header = True, index = True)

## 2. Por categorías específicas

Con la lista de mitologías, se buscan las categorías disponibles en Wikipedia que únicamente contengan personajes mitológicos de todas estas mitologías. Algunas categorías son únicas para cada mitología en particular, así que se añaden a mano.

In [3]:
mitologias = pd.read_csv("mitologias.csv")
mitologias

,Mitología,Region,Continent
0,Mitología lugbara,Central Africa,Africa
1,Mitología egipcia,North Africa,Africa
2,Mitología china,Northeast Asia,Asia
3,Mitología japonesa,Northeast Asia,Asia
4,Mitología coreana,Northeast Asia,Asia
...,...,...,...
210,Mitología yoruba,West Africa,Africa
211,Mitología zulú,Southern Africa,Africa
212,Mitología zuñi,North America,Americas
213,Mitología ática,Southern Europe,Europe


### Obtención de las URLs de las categorías

Primero se obtiene un dataset con todas las URLs de las categorías que scrapearemos más adelante.

Las URLs de las categorías genéricas están formadas por (1) el *tipo de categoría* y (2) la *mitología*.

Ejemplo: `https://es.wikipedia.org/wiki/Categoría:Dioses_griegos` , donde `Dioses` es la categoría y `griegos` es la mitología.

1. Existen diferentes categorías que contienen personajes mitológicos (Deidades, Dioses, Gigantes, Demonios, Dragones ...).  
    Cada mitología tiene sus propias categorías, por lo que he elegido las categorías que he considerado compartidas entre mitologías. Estas categorías son: Personaje, Deidad, Dios, Diosa, Criatura, Dragón, Gigante, Demonio, Monstruo y Héroe.  
    Por otro lado, cada categoría puede tener diversos formatos según la mitología en particular. Por ejemplo, la categoría de "Criatura" puede estar formateada como: `Criaturas`, `Criaturas_mitológicas` y `Criaturas_de_la_mitología`.

In [4]:
# anteriormente se han probado más formatos
categs = {"Personajes_de_la_mitología" : "Personaje", "Deidades" : "Deidad", "Dioses_de_la_mitología" : "Dios",
          "Dioses" : "Dios", "Diosas_de_la_mitología" : "Diosa", "Diosas" : "Diosa", "Criaturas_de_la_mitología" : "Criatura",
          "Criaturas_mitológicas" : "Criatura", "Dragones" : "Dragón", "Gigantes_de_la_mitología" : "Gigante"}

2. El formato de la mitología depende tanto del tipo/formato de la categoría como de la propia mitología.  
    Por ejemplo, en la mitología egipcia podemos tener `Dioses_egipcios`, `Diosas_egipcias` y `Deidades_de_la_mitología_egipcia`. En cada caso, el formato de la mitología es diferente.  
    Además, cada mitología tiene su particular transformación cuando cambia de género y número. Los sustantivos del español suelen seguir unas reglas para esta transformación, aunque existen excepciones. Se aplican las 5 reglas más comunes a las mitologías para obtener sus términos clave, por lo que probablemente una de estas claves será correcta.  
    Algunas URLs tienen mayúscula la primera letra de la mitología, por lo que añado estas posibilidades a las claves.

Función para obtener los términos clave de una mitología.

In [5]:
def clave_mitologia(mit):
    lista = []
    lista.append("_".join(mit.split()[1:])) # elemento 0 en minuscula
    lista.append("_".join(mit.split()[1:-1] + [mit.split()[-1].capitalize()])) # elemento 1 en mayuscula
    for i in range(2):
        lista.append(lista[i] + "s")
        lista.append(lista[i] + "es")
        lista.append(lista[i][0:-1] + "os")
        lista.append(lista[i][0:-1] + "es")
    return(lista)

Función para comprobar si la URL de la categoría contiene personajes.

In [6]:
def url_buena(url):
    doc = get_url(url)
    if doc != None:
        paginas = doc.find("div", id = "mw-pages")
        if paginas != None:
            return(True)
    return(False)

Función para obtener las posibles URLs de una mitología.

In [7]:
def obtener_urls(mit):
    url_base = "https://es.wikipedia.org/wiki/Categoría:{}_"
    urls = pd.DataFrame()
    keys = clave_mitologia(mit)
    formatos = list(categs.keys())
    for formato in formatos:
        for key in keys:
            url = url_base.format(formato) + key
            if url_buena(url):
                fila = pd.DataFrame([[url,
                                      mit,
                                      categs[formato]]])
                urls = urls.append(fila)
    return(urls.drop_duplicates(ignore_index = True))

Se combinan los tipos de categoría y las mitologías y se obtiene un dataset con las categorías existentes en Wikipedia.

**Tarda mucho en ejecutarse**

In [8]:
urls_categorias = pd.DataFrame()

for mitologia in mitologias["Mitología"]:
    urls_categorias = urls_categorias.append(obtener_urls(mitologia))

urls_categorias.columns = ["URL", "Mitología", "Clase"]

Se añaden directamente las siguientes categorías con formato único:
* [Dioses primordiales de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega)
* [Titanes (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Titanes)
* [Deidades de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega)
* [Dioses olímpicos menores de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores)
* [Héroes de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega)
* [Monstruos de la mitología griega](https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega)
* [Musas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Musas)
* [Oceánidas (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánidas)
* [Oceánides (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Oceánides)
* [Descendientes de Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus)
* [Descendientes de Poseidón (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón)
* [Descendientes de Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo)
* [Descendientes de Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita)
* [Diosas tomadas por Zeus (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus)
* [Diosas amadas por Apolo (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo)
* [Dioses que tomaron a Afrodita (mitología griega)](https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita)
* [Jotuns (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Jotuns)
* [Vanir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Vanir)
* [Æsir (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Æsir)
* [Ásynjur (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Ásynjur)
* [Nornas (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Nornas)
* [Lobos de la mitología nórdica](https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica)
* [Hijos de Odín (mitología nórdica)](https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín)
* [Demonios de la mitología hinduista](https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo)
* [Divinidades sintoistas (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas)
* [Yōkai (mitología japonesa)](https://es.wikipedia.org/wiki/Categoría:Yōkai)
* [Tonalteteuctin (mitología mexica)](https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin)
* [Dioses del viento de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica)
* [Criaturas de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas)
* [Demonios de la mitología mexica](https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica)
* [Dharmapalas (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dharmapalas)
* [Dakinis (mitología budista)](https://es.wikipedia.org/wiki/Categoría:Dakinis)
* [Deidades del budismo tibetano](https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano)
* [Seres de la mitología budista](https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista)
* [Demonios de la mitología budista](https://es.wikipedia.org/wiki/Categoría:Demonios_budistas)
* [Ráksasas de la mitología del hinduismo y budista](https://es.wikipedia.org/wiki/Categoría:Ráksasas)

In [9]:
addCategs = [["https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Titanes","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Héroes_de_la_mitología_griega","Mitología griega","Héroe"],
             ["https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega","Mitología griega","Monstruo"],
             ["https://es.wikipedia.org/wiki/Categoría:Musas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánidas","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánides","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Zeus","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Poseidón","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Apolo","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Descendientes_de_Afrodita","Mitología griega","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Zeus","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_amadas_por_Apolo","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_que_tomaron_a_Afrodita","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Jotuns","Mitología nórdica","Gigante"],
             ["https://es.wikipedia.org/wiki/Categoría:Vanir","Mitología nórdica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Æsir","Mitología nórdica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Ásynjur","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nornas","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Hijos_de_Odín","Mitología nórdica","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo","Mitología del hinduismo","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas","Mitología japonesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Yōkai","Mitología japonesa","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas","Mitología mexica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_de_la_mitología_mexica","Mitología mexica","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Dharmapalas","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dakinis","Mitología budista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista","Mitología budista","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_budistas","Mitología budista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología del hinduismo","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología budista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Orisha","Mitología yoruba","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tlaloque","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Centzon_Totochtin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tres_augustos_y_cinco_emperadores","Mitología china","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_tomadas_por_Poseidón","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nereidas","Mitología griega","Deidad"]]

urls_categorias = urls_categorias.append(pd.DataFrame(addCategs, columns = ["URL", "Mitología", "Clase"]))

Se añaden las URLs de las categorías con más de una página de personajes.

Función para obtener la URL de la siguiente página de personajes en el caso de que exista.

In [10]:
def obtener_siguiente(pagina):
    first_link = pagina.find("div", id = "mw-pages").find("a")
    if first_link.text == "página siguiente":
        return "https://es.wikipedia.org" + first_link.get("href")
    elif first_link.text == "página anterior":
        second_link = pagina.find("div", id = "mw-pages").find_all("a")[1]
        if second_link.text == "página siguiente":
            return "https://es.wikipedia.org" + second_link.get("href")

In [11]:
urls_categorias = urls_categorias.reset_index(drop = True)

for i in range(len(urls_categorias)):
    next_url = obtener_siguiente(get_url(urls_categorias["URL"][i]))
    while next_url is not None:
        mit = urls_categorias["Mitología"][i]
        new_categoria = pd.DataFrame([[next_url, mit, urls_categorias["Clase"][i]]])
        new_categoria.columns = ["URL", "Mitología", "Clase"]
        urls_categorias = urls_categorias.append(new_categoria)
        next_url = obtener_siguiente(get_url(next_url))

In [12]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología egipcia,Diosa
2,https://es.wikipedia.org/wiki/Categoría:Criatu...,Mitología egipcia,Criatura
3,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología china,Dios
4,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología china,Diosa
...,...,...,...
126,https://es.wikipedia.org/wiki/Categoría:Tres_a...,Mitología china,Personaje
127,https://es.wikipedia.org/wiki/Categoría:Diosas...,Mitología griega,Diosa
128,https://es.wikipedia.org/wiki/Categoría:Nereidas,Mitología griega,Deidad
0,https://es.wikipedia.org/w/index.php?title=Cat...,Mitología griega,Personaje


### Obtención de las URLs de los personajes

Ahora se obtiene un dataset con las URLs de los personajes pertenecientes a las categorías extraídas en el subapartado anterior. También se añaden algunos personajes importantes manualmente.

Función para limpiar el nombre del personaje.

In [13]:
def clean_name(nombre):
    if nombre.find("mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("Mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("(dios") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("deidad") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("personaje") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    else:
        return(nombre)

Función para obtener los nombres y las URLs de los personajes de la categoría.  
La división alfabética de los personajes puede estar en dos formatos diferentes:  

* Todas las categorias en el mismo nodo
* Cada categoria tiene su nodo

In [14]:
def scrap_categoria(url):
    pagina = get_url(url)
    lista_pers = []
    lista_urls = []
    div_alfabetica = pagina.find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr")
    if div_alfabetica.find("div", class_ = "mw-category") is None: # caso 1
        categorias = div_alfabetica.find_all("h3")
        check_letras = [categoria.text.isalpha() for categoria in categorias]
        personajes = div_alfabetica.find_all("ul")
        for i in range(len(categorias)):
            if check_letras[i]:
                pers = personajes[i]
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})
    else: # caso 2
        categorias = div_alfabetica.find_all("div", class_ = "mw-category-group")
        for categoria in categorias:
            if categoria.find("h3").text.isalpha():
                pers = categoria.find_all("li")
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})

Se obtienen los personajes de las categorías con sus URLs.

In [15]:
urls_personajes1 = pd.DataFrame()

for url in urls_categorias["URL"]:
    urls_personajes1 = urls_personajes1.append(scrap_categoria(url))

In [16]:
urls_personajes1

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
21,https://es.wikipedia.org/wiki/Yaso_(desambigua...,Yaso (desambiguación),Mitología griega,Personaje
22,https://es.wikipedia.org/wiki/Y%C3%B3bates,Yóbates,Mitología griega,Personaje
23,https://es.wikipedia.org/wiki/Yolao,Yolao,Mitología griega,Personaje
24,https://es.wikipedia.org/wiki/Zacinto_(mitolog...,Zacinto,Mitología griega,Personaje


In [17]:
urls_personajes1.to_csv("urls_pers1.csv", index = False)

## 3. Por intersección de categorías

Ahora se obtendrán los artículos que se encuentren en la categoría de alguna mitología y a su vez en una categoría de personaje mitológico. Por ejemplo: se obtiene el personaje "Fenghuang" porque se encuentra en las categorías "Mitología china" y "Aves legendarias".

In [18]:
mitologias["Mitología"]

0       Mitología lugbara
1       Mitología egipcia
2         Mitología china
3      Mitología japonesa
4       Mitología coreana
              ...        
210      Mitología yoruba
211        Mitología zulú
212        Mitología zuñi
213       Mitología ática
214       Mitología íbera
Name: Mitología, Length: 215, dtype: object

Se obtienen las páginas de las categorías disponibles de cada mitología.

In [19]:
lista_url_mits = []

for mitologia in mitologias["Mitología"]:
    url = "https://es.wikipedia.org/wiki/Categoría:" + mitologia.replace(" ","_")
    if url_buena(url):
        lista_url_mits.append(url)
        next_url = obtener_siguiente(get_url(url))
        while next_url is not None:
            lista_url_mits.append(next_url)
            next_url = obtener_siguiente(get_url(next_url))

urls_mits = (pd
             .DataFrame(
                 list(mitologias["Mitología"]),
                 index = ["https://es.wikipedia.org/wiki/Categoría:" + m.replace(" ","_") for m in list(mitologias["Mitología"])])
             .loc[lista_url_mits])
urls_mits.columns = ["Mitología"]

urls_mits

,Mitología
https://es.wikipedia.org/wiki/Categoría:Mitología_egipcia,Mitología egipcia
https://es.wikipedia.org/wiki/Categoría:Mitología_china,Mitología china
https://es.wikipedia.org/wiki/Categoría:Mitología_japonesa,Mitología japonesa
https://es.wikipedia.org/wiki/Categoría:Mitología_coreana,Mitología coreana
https://es.wikipedia.org/wiki/Categoría:Mitología_túrquica,Mitología túrquica
...,...
https://es.wikipedia.org/wiki/Categoría:Mitología_venezolana,Mitología venezolana
https://es.wikipedia.org/wiki/Categoría:Mitología_yoruba,Mitología yoruba
https://es.wikipedia.org/wiki/Categoría:Mitología_zulú,Mitología zulú
https://es.wikipedia.org/wiki/Categoría:Mitología_ática,Mitología ática


Se obtienen todos los artículos de las categorías de mitologías.

In [20]:
lista_urls = []
lista_pers = []
lista_mits = []

for url in urls_mits.index:
    nodos = get_url(url).find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr").find_all("a")
    lista_urls = lista_urls + ["https://es.wikipedia.org" + nodo.get("href") for nodo in nodos]
    lista_pers = lista_pers + [clean_name(nodo.text) for nodo in nodos]
    lista_mits = lista_mits + len(nodos) * [urls_mits.loc[url]["Mitología"]]

articulos_mits = pd.DataFrame({"URL" : lista_urls, "Nombre" : lista_pers, "Mitología" : lista_mits})

articulos_mits

,URL,Nombre,Mitología
0,https://es.wikipedia.org/wiki/Mitolog%C3%ADa_e...,Mitología egipci,Mitología egipcia
1,https://es.wikipedia.org/wiki/Aaru,Aaru,Mitología egipcia
2,https://es.wikipedia.org/wiki/Abatos,Abatos,Mitología egipcia
3,https://es.wikipedia.org/wiki/Aj_(mitolog%C3%ADa),Aj,Mitología egipcia
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia
...,...,...,...
2518,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera
2519,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera
2520,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera
2521,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera


Función para obtener las categorías de un artículo.

In [21]:
def cats_articulo(url):
    art_cats = get_url(url).find("div", id = "mw-normal-catlinks").find("ul").find_all("a")
    return [cats.text for cats in art_cats]

Funciones para comprobar si los artículos pertenecen a categorías de personajes mitológicos.

In [22]:
def check_deidad(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Deidad", "Dios"])])
    return(check)

In [23]:
def check_criatura(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Criatura", "Animales mitológicos", "Arañas mitológicas", "Aves legendarias",
                                                 "Bovinos mitológicos", "Caballos mitológicos", "Caprinos mitológicos",
                                                 "Felinos mitológicos", "Gatos mitológicos", "Leones mitológicos",
                                                 "Peces mitológicos", "Perros mitológicos", "Reptiles mitológicos",
                                                 "Serpientes mitológicas", "Críptido", "Cambiante", "Quimera"])])
    return(check)

In [24]:
def check_dragon(lista):
    check = False
    for l in lista:
        check = any([check, "Dragones" in l])
    return(check)

In [25]:
def check_gigante(lista):
    check = False
    for l in lista:
        check = any([check, "Gigante" in l])
    return(check)

In [26]:
def check_heroe(lista):
    check = False
    for l in lista:
        check = any([check, "Héroe" in l])
    return(check)

In [27]:
def check_monstruo(lista):
    check = False
    for l in lista:
        check = any([check, "Monstruo" in l])
    return(check)

In [28]:
def check_demonio(lista):
    check = False
    for l in lista:
        check = any([check, "Demonio" in l])
    return(check)

De los artículos anteriores únicamente obtenemos los personajes mitológicos. Es decir, los que pertenecen a categorías que incluyen la palabra "Dios", "Monstruo", "Demonio",...

In [29]:
urls_personajes2 = pd.DataFrame()

for i in range(len(articulos_mits)):
    lista_cats = cats_articulo(articulos_mits["URL"][i])
    if check_deidad(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Deidad"]])))
    if check_criatura(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Criatura"]])))
    if check_dragon(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Dragón"]])))
    if check_gigante(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Gigante"]])))
    if check_heroe(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Héroe"]])))
    if check_monstruo(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Monstruo"]])))
    if check_demonio(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Demonio"]])))

urls_personajes2.columns = ["URL", "Nombre", "Mitología", "Clase"]

In [30]:
urls_personajes2

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Deidad
0,https://es.wikipedia.org/wiki/Bujis,Bujis,Mitología egipcia,Criatura
0,https://es.wikipedia.org/wiki/Cuervo_de_tres_p...,Cuervo de tres patas,Mitología egipcia,Criatura
0,https://es.wikipedia.org/wiki/Neper_(mitolog%C...,Neper,Mitología egipcia,Deidad
0,https://es.wikipedia.org/wiki/Nun_(mitolog%C3%...,Nun,Mitología egipcia,Deidad
...,...,...,...,...
0,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


In [31]:
urls_personajes2.to_csv("urls_pers2.csv", index = False)

## 4. Dataset de personajes

Se unen los dos datasets anteriores.

In [3]:
url_pers = pd.read_csv("urls_pers1.csv").append(pd.read_csv("urls_pers2.csv"))
url_pers

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
1116,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
1117,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
1118,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
1119,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


Se añaden directamente algunos personajes mitológicos importantes faltantes:

*Mitología griega*
* [Tánatos](https://es.wikipedia.org/wiki/Tánatos)
* [Zeus](https://es.wikipedia.org/wiki/Zeus)
* [Poseidón](https://es.wikipedia.org/wiki/Poseidón)
* [Apolo](https://es.wikipedia.org/wiki/Apolo)
* [Atenea](https://es.wikipedia.org/wiki/Atenea)
* [Artemisa](https://es.wikipedia.org/wiki/Artemisa)
* [Deméter](https://es.wikipedia.org/wiki/Deméter)
* [Heracles](https://es.wikipedia.org/wiki/Heracles)
* [Teseo](https://es.wikipedia.org/wiki/Teseo)
* [Jasón](https://es.wikipedia.org/wiki/Jasón)

*Mitología romana*
* [Mitra](https://es.wikipedia.org/wiki/Mitra_(dios_romano))
* [Magna Mater](https://es.wikipedia.org/wiki/Magna_Mater)

*Mitología nórdica*
* [Hela](https://es.wikipedia.org/wiki/Hela)

*Mitología mesopotámica*
* [Gilgamesh](https://es.wikipedia.org/wiki/Gilgamesh)

*Mitología sumeria*
* [Anunnaki](https://es.wikipedia.org/wiki/Anunnaki)

*Mitología incaica*
* [Cuchavira](https://es.wikipedia.org/wiki/Cuchavira)

*Mitología pueblo*
* [Awelo](https://es.wikipedia.org/wiki/Awelo)

entre otros...

In [4]:
new_pers = [["https://es.wikipedia.org/wiki/Tánatos","Tánatos","Mitología griega","Deidad"],
           ["https://es.wikipedia.org/wiki/Zeus","Zeus","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Poseidón","Poseidón","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Apolo","Apolo","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Atenea","Atenea","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Artemisa","Artemisa","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Deméter","Deméter","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Heracles","Heracles","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Teseo","Teseo","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Jasón","Jasón","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Mitra_(dios_romano)","Mitra","Mitología romana","Dios"],
           ["https://es.wikipedia.org/wiki/Magna_Mater","Magna Mater","Mitología romana","Diosa"],
           ["https://es.wikipedia.org/wiki/Hela","Hela","Mitología nórdica","Deidad"],
           ["https://es.wikipedia.org/wiki/Gilgamesh","Gilgamesh","Mitología mesopotámica","Héroe"],
           ["https://es.wikipedia.org/wiki/Anunnaki", "Anunnaki", "Mitología sumeria", "Deidad"],
           ["https://es.wikipedia.org/wiki/Cuchavira","Cuchavira","Mitología incaica","Dios"],
           ["https://es.wikipedia.org/wiki/Awelo","Awelo","Mitología pueblo","Dios"],
           ["https://es.wikipedia.org/wiki/Asase_Ya","Asase Ya","Mitología africana","Diosa"],
           ["https://es.wikipedia.org/wiki/Xólotl_(divinidad)","Xólotl","Mitología mexica","Dios"],
           ["https://es.wikipedia.org/wiki/Chicomecóatl","Chicomecóatl","Mitología mexica","Diosa"],
           ["https://es.wikipedia.org/wiki/Macuiltochtli","Macuiltochtli","Mitología mexica","Dios"],
           ["https://es.wikipedia.org/wiki/Xochiquétzal","Xochiquétzal","Mitología mexica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ñamandú","Ñamandú","Mitología guaraní","Dios"],
           ["https://es.wikipedia.org/wiki/Lahmu","Lahmu","Mitología mesopotámica","Gigante"],
           ["https://es.wikipedia.org/wiki/Mummu","Mummu","Mitología mesopotámica","Deidad"],
           ["https://es.wikipedia.org/wiki/Enkimdu","Enkimdu","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Damkina","Damkina","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Emesh","Emesh","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Enbilulu","Enbilulu","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Enten","Enten","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Erra","Erra","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Geshtu-e","Geshtu-e","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Gugalanna","Gugalanna","Mitología sumeria","Deidad"],
           ["https://es.wikipedia.org/wiki/Isimud","Isimud","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Musdamma","Musdamma","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Namtar","Namtar","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Ninazu","Ninazu","Mitología sumeria","Dios"],
           ["https://es.wikipedia.org/wiki/Ninkasi","Ninkasi","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninkur","Ninkur","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninlil","Ninlil","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsar","Ninsar","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsikila","Ninsikila","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Ninsuna","Ninsuna","Mitología mesopotámica","Diosa"],
           ["https://es.wikipedia.org/wiki/Antum","Antum","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Nusku","Nusku","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Tišpak","Tišpak","Mitología mesopotámica","Dios"],
           ["https://es.wikipedia.org/wiki/Uras","Uras","Mitología sumeria","Diosa"],
           ["https://es.wikipedia.org/wiki/Emperador_Ku","Emperador Ku","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Shaohao","Shaohao","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Zhuanxu","Zhuanxu","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Emperador_Yao","Emperador Yao","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Emperador_Shun","Emperador Shun","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Yu_el_Grande","Yu el Grande","Mitología china","Personaje"],
           ["https://es.wikipedia.org/wiki/Fushoulu","Fushoulu","Mitología china","Dios"],
           ["https://es.wikipedia.org/wiki/Sun_Wukong","Sun Wukong","Mitología china","Héroe"],
           ["https://es.wikipedia.org/wiki/Bóreas","Bóreas","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Céfiro","Céfiro","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Cibeles","Cibeles","Mitología griega","Diosa"]]

personajes = url_pers.append(pd.DataFrame(new_pers, columns = ["URL", "Nombre", "Mitología", "Clase"])).reset_index(drop = True)

personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios
...,...,...,...,...
4310,https://es.wikipedia.org/wiki/Fushoulu,Fushoulu,Mitología china,Dios
4311,https://es.wikipedia.org/wiki/Sun_Wukong,Sun Wukong,Mitología china,Héroe
4312,https://es.wikipedia.org/wiki/Bóreas,Bóreas,Mitología griega,Dios
4313,https://es.wikipedia.org/wiki/Céfiro,Céfiro,Mitología griega,Dios


### Unión de personajes con región

In [5]:
mitologias = pd.read_csv("mitologias.csv")

mitologias

,Mitología,Region,Continent
0,Mitología lugbara,Central Africa,Africa
1,Mitología egipcia,North Africa,Africa
2,Mitología china,Northeast Asia,Asia
3,Mitología japonesa,Northeast Asia,Asia
4,Mitología coreana,Northeast Asia,Asia
...,...,...,...
210,Mitología yoruba,West Africa,Africa
211,Mitología zulú,Southern Africa,Africa
212,Mitología zuñi,North America,Americas
213,Mitología ática,Southern Europe,Europe


In [6]:
personajes["Region"] = [mitologias.loc[mitologias["Mitología"] == mit].iloc[0]["Region"] for mit in personajes["Mitología"]]
personajes["Continent"] = [mitologias.loc[mitologias["Mitología"] == mit].iloc[0]["Continent"] for mit in personajes["Mitología"]]

personajes

,URL,Nombre,Mitología,Clase,Region,Continent
0,https://es.wikipedia.org/wiki/Aken_(dios),Aken,Mitología egipcia,Dios,North Africa,Africa
1,https://es.wikipedia.org/wiki/Aker,Aker,Mitología egipcia,Dios,North Africa,Africa
2,https://es.wikipedia.org/wiki/Amenhotep_(hijo_...,Amenhotep (hijo de Hapu),Mitología egipcia,Dios,North Africa,Africa
3,https://es.wikipedia.org/wiki/Am%C3%B3n,Amón,Mitología egipcia,Dios,North Africa,Africa
4,https://es.wikipedia.org/wiki/Amset,Amset,Mitología egipcia,Dios,North Africa,Africa
...,...,...,...,...,...,...
4310,https://es.wikipedia.org/wiki/Fushoulu,Fushoulu,Mitología china,Dios,Northeast Asia,Asia
4311,https://es.wikipedia.org/wiki/Sun_Wukong,Sun Wukong,Mitología china,Héroe,Northeast Asia,Asia
4312,https://es.wikipedia.org/wiki/Bóreas,Bóreas,Mitología griega,Dios,Southern Europe,Europe
4313,https://es.wikipedia.org/wiki/Céfiro,Céfiro,Mitología griega,Dios,Southern Europe,Europe


### One-hot Encoding

Hay personajes repetidos, es decir, personajes que pertenecen a distintas Mitologías o a distintas Clases. Por lo tanto, cambiamos el formato de las variables "Mitología", "Clase", "Region" y "Continent" a One-Hot Encoding.

In [7]:
personajes.groupby(by = ["URL", "Nombre"]).size().sort_values(ascending = False)

URL                                                            Nombre               
https://es.wikipedia.org/wiki/Humbaba                          Humbaba                  12
https://es.wikipedia.org/wiki/Enlil                            Enlil                    11
https://es.wikipedia.org/wiki/Baal                             Baal                      8
https://es.wikipedia.org/wiki/Siguanaba                        Siguanaba                 8
https://es.wikipedia.org/wiki/El_(dios_sem%C3%ADtico)          El                        8
                                                                                        ..
https://es.wikipedia.org/wiki/Marraco                          Marraco                   1
https://es.wikipedia.org/wiki/Maristino                        Maristino                 1
https://es.wikipedia.org/wiki/Maris_(mitolog%C3%ADa)           Maris                     1
https://es.wikipedia.org/wiki/Mari_(diosa)                     Mari                      1
https

In [8]:
ohe = (pd
       .get_dummies(personajes[["Mitología", "Clase", "Region", "Continent"]],
                    prefix = {"Mitología" : "", "Clase" : "Clase", "Region" : "Region", "Continent" : "Continent"},
                    prefix_sep = {"Mitología" : "", "Clase" : ":", "Region" : ":", "Continent" : ":"}))
personajes = (pd.
        concat([personajes, ohe], axis = 1).
        drop(columns = ["Mitología", "Clase", "Clase:Personaje", "Region", "Continent"]).
        groupby(by = ["URL", "Nombre"]).
        agg(max).
        reset_index())

personajes

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,https://es.wikipedia.org/wiki/%C3%81gave_(mito...,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3174,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3175,https://es.wikipedia.org/wiki/Zorro_de_nueve_c...,Zorro de nueve colas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3176,https://es.wikipedia.org/wiki/Zurvan,Zurvan,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


### Limpieza

In [9]:
personajes.groupby("Nombre").size().sort_values(ascending = False)

Nombre
Cloris              2
Mitra               2
Anu                 2
Ágave               2
Aletes              2
                   ..
Nike                1
Nictimene           1
Nibiru (planeta)    1
Nian                1
Aba                 1
Length: 3173, dtype: int64

"https://es.wikipedia.org/wiki/Cloris" es desambiguación

In [13]:
personajes[personajes["Nombre"] == "Cloris"]

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
755,https://es.wikipedia.org/wiki/Cloris,Cloris,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
757,https://es.wikipedia.org/wiki/Cloris_(mitolog%...,Cloris,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


Hay 2 Mitra: un dios indio y un dios romano

In [14]:
personajes[personajes["Nombre"] == "Mitra"]

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
2053,https://es.wikipedia.org/wiki/Mitra_(dios_indio),Mitra,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2054,https://es.wikipedia.org/wiki/Mitra_(dios_romano),Mitra,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [16]:
personajes.loc[personajes.URL == "https://es.wikipedia.org/wiki/Mitra_(dios_indio)", "Nombre"] = "Mitra (mitología del hinduismo)"
personajes.loc[personajes.URL == "https://es.wikipedia.org/wiki/Mitra_(dios_romano)", "Nombre"] = "Mitra (mitología romana)"

Hay 2 Anu: dios sumerio y diosa celta

In [17]:
personajes[personajes["Nombre"] == "Anu"]

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
299,https://es.wikipedia.org/wiki/Anu,Anu,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
300,https://es.wikipedia.org/wiki/Anu_(diosa_celta),Anu,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [18]:
personajes.loc[personajes.URL == "https://es.wikipedia.org/wiki/Anu", "Nombre"] = "Anu (mitología mesopotámica)"
personajes.loc[personajes.URL == "https://es.wikipedia.org/wiki/Anu_(diosa_celta)", "Nombre"] = "Anu (mitología celta)"

"https://es.wikipedia.org/wiki/Ágave_(mitología)" es desambiguación

In [19]:
personajes[personajes["Nombre"] == "Ágave"]

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,https://es.wikipedia.org/wiki/%C3%81gave_(mito...,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


"https://es.wikipedia.org/wiki/Aletes_(deidad)" es un personaje histórico, asi que se quita

In [20]:
personajes[personajes["Nombre"] == "Aletes"]

,URL,Nombre,Mitología Báltica,Mitología aborigen australiana,Mitología abrahámica,Mitología acadia,Mitología africana,Mitología afroamericana,Mitología aimara,Mitología amazónica,...,Region:Southern Africa,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
188,https://es.wikipedia.org/wiki/Aletes,Aletes,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
189,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [21]:
personajes = personajes[personajes["URL"] != "https://es.wikipedia.org/wiki/Aletes_(deidad)"]

In [43]:
urls_personajes = urls_personajes[[not("desambiguación" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("mitología" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("dios" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("Guerra" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[[not("Anexo" in pers) for pers in list(urls_personajes["Nombre"])]]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/Cloris"]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/%C3%81gave_(mitolog%C3%ADa)"]
urls_personajes = urls_personajes[urls_personajes["URL"] != "https://es.wikipedia.org/wiki/Aletes_(deidad)"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses Olímpicos (DC Comics)"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses domésticos"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Dioses egipcios"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Diosas Tenma"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Siete Dioses de la Fortuna"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Deidades feroces"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Estatua de Sejmet"]
urls_personajes = urls_personajes[urls_personajes["Nombre"] != "Divinidades hinduistas"]

In [45]:
personajes.groupby("Nombre").size().sort_values(ascending = False)

Nombre
Šakkan                            1
Esperqueo                         1
Euclea                            1
Eubuleo                           1
Etón                              1
                                 ..
Neso                              1
Nerthus                           1
Nerites                           1
Neptuno                           1
Abante (compañero de Diomedes)    1
Length: 2596, dtype: int64

In [46]:
personajes

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Region:South Asia,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
5,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2619,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2620,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2621,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [47]:
personajes.to_csv("personajes.csv", header = True, index = False)

## 4. Obtención texto de personajes

In [180]:
data = pd.read_csv("urls_personajes.csv")
data

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Region:South Asia,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2592,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2593,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2594,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


Obtenemos el texto de cada artículo.

Función para limpiar el texto.

In [21]:
def clean_text(text):
    text = re.sub("[^A-Za-zÀ-ÿ\u00f1\u00d1]", " ", text)
    text = text.lower()
    out = ""
    for word in text.split():
        out += " " + word
    return(out.strip())

Función para obtener todo el texto útil del artículo de wikipedia. Se omiten los apartados de "Enlaces externos", "Referencias", "Fuentes", "Notas", "Véase también" y "Bibliografía".

In [22]:
omit = ["enlaces externos", "referencias", "fuentes", "notas", "véase también", "bibliografía"]

def obt_texto(url):
    pag = get_url(url)
    nodos = pag.find("div", class_ = "mw-parser-output").contents
    texto = ""
    h2 = True
    h3 = True
    for i in range(len(nodos)):
        if isinstance(nodos[i], str):
            continue
        if nodos[i].get("class") is not None:
            if ["noprint", "VT rellink", "ext-quick-survey-panel"] in nodos[i].get("class") or "VT ":
                continue
        tipo = nodos[i].name
        if tipo == "h2":
            if len(set([nodos[i].text.lower().find(o) for o in omit])) == 1: # si el string no contiene ninguna de las palabras clave...
                h2 = True
                h3 = True
            else: # si el string contiene alguna de las palabras que se quieren omitir...
                h2 = False
        if tipo == "h3":
            if len(set([nodos[i].text.find(o) for o in omit])) == 1:
                h3 = True
            else:
                h3 = False
        if h2 and h3:
            texto += nodos[i].text
    return(clean_text(texto))

Función para obtener los links dirigidos a otros personajes.

In [23]:
def row_matriz_ady(url):
    row = [0] * len(data)
    a_list = get_url(url).find("div", class_ = "mw-parser-output").find_all("a")
    links = ["https://es.wikipedia.org" + str(link.get("href")) for link in a_list]
    for link in links:
        if link in list(data["URL"]):
            row[data.index[data["URL"] == link][0]] = 1
    df = pd.DataFrame([row], index = [url], columns = data["Nombre"])
    return df

In [52]:
textos = []
ady_matrix = pd.DataFrame()
for url in data["URL"]:
    textos.append(obt_texto(url))
    ady_matrix = ady_matrix.append(row_matriz_ady(url))

data["Texto"] = textos

data

,URL,Nombre,Mitología africana,Mitología americana,Mitología asiática,Mitología budista,Mitología cananea,Mitología celta,Mitología china,Mitología coreana,...,Region:Southern Europe,Region:West Africa,Region:West Asia / Middle East,Region:Western Europe,Continent:Africa,Continent:Americas,Continent:Asia,Continent:Europe,Continent:Oceania,Texto
0,https://es.wikipedia.org/wiki/%C3%81ctor_(hijo...,Áctor (hijo de Deión),0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,en la mitología griega áctor era padre de mene...
1,https://es.wikipedia.org/wiki/%C3%81ctor_(rey_...,Áctor (rey de Ftía),0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,en la mitología griega áctor en griego antiguo...
2,https://es.wikipedia.org/wiki/%C3%81diti,Áditi,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,en el marco del vedismo áditi es una diosa mad...
3,https://es.wikipedia.org/wiki/%C3%81gave,Ágave,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,en la mitología griega ágave en griego antiguo...
4,https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%...,Álfröðull,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,álfröðull alfrodhul del nórdico antiguo gloria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,https://es.wikipedia.org/wiki/Zeuxipo,Zeuxipo,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,en la mitología griega zeuxipo griego antiguo ...
2592,https://es.wikipedia.org/wiki/Zipacn%C3%A1,Zipacná,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,zipacná era en la mitología maya hijo de vucub...
2593,https://es.wikipedia.org/wiki/Zonget,Zonget,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,zonget es la diosa de la naturaleza en la mito...
2594,https://es.wikipedia.org/wiki/Zorra_teumesia,Zorra teumesia,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,en la mitología griega la zorra teumesia en gr...


In [56]:
ady_matrix

Nombre,Áctor (hijo de Deión),Áctor (rey de Ftía),Áditi,Ágave,Álfröðull,Álope,Ámbrax,Ámico,Ántifo (hijo de Mirmidón),Ápate,...,Zao Shen,Zashiki-warashi,Zelo,Zetes,Zeus,Zeuxipo,Zipacná,Zonget,Zorra teumesia,Zorro de nueve colas
https://es.wikipedia.org/wiki/%C3%81ctor_(hijo_de_Dei%C3%B3n),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81ctor_(rey_de_Ft%C3%ADa),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81diti,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81gave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
https://es.wikipedia.org/wiki/%C3%81lfr%C3%B6%C3%B0ull,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://es.wikipedia.org/wiki/Zeuxipo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zipacn%C3%A1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zonget,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Zorra_teumesia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [57]:
data.to_csv("data.csv", header = True, index = False)
ady_matrix.to_csv("ady_matrix.csv", header = True, index = False)